In [1]:
import os
from openai import OpenAI
import cohere

In [9]:
co = cohere.Client(os.getenv('COHERE_API_KEY'))
openai_client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY")
)

In [10]:
def test_prompt_openai(prompt, suppress=False, model='gpt-3.5-turbo', **kwargs):
    " a simple function to take in a prompt and run it through a given non-chat model "
        
    chat_completion = openai_client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
        **kwargs
    )
    answer = chat_completion.choices[0].message.content
    if not suppress:
        print(f'PROMPT:\n------\n{prompt}\n------\nRESPONSE\n------\n{answer}')
    else:
        return answer


In [14]:
def test_prompt_cohere(prompt, suppress=False, model='command-xlarge', **kwargs):
    response = co.generate(
        model=model,
        prompt=prompt,
        **kwargs,
      )
    if not suppress:
        print(f'PROMPT:\n------\n{prompt}\n------\nRESPONSE\n------\n{response.generations[0].text}')

## Just ASK

In [15]:
test_prompt_openai('Translate to Turkish.\n\nWhere is the nearest restaurant?')

PROMPT:
------
Translate to Turkish.

Where is the nearest restaurant?
------
RESPONSE
------
En yakın restoran nerede?


In [16]:
test_prompt_cohere('Translate to Turkish.\n\nWhere is the nearest restaurant?')

PROMPT:
------
Translate to Turkish.

Where is the nearest restaurant?
------
RESPONSE
------
 Turkish:

Nearest restaurant's location in Turkish would be:

En iyi restoranın en yakınında nerede?

This translation asks "Where is the nearest restaurant located?" en iyi restoran means "the best restaurant," but in this context, it asks about a restaurant in general; en yakınında modifies nerede, which means "where" or "at which place?"

Let me know if I can be of further assistance regarding Turkish translation. 


# Retrieval Augmented Generation with GPT-4

In [51]:
SYSTEM_PROMPT = '''
You are a helpful Q/A bot that can only use reference material from a knowledge base.
If a user asks anything that is not "from the knowledge base", say that you cannot answer.
'''.strip()

class ChatbotGPT():
    def __init__(self, system_prompt, threshold=.8):
        self.conversation = [{'role': 'system', 'content': system_prompt}]
        self.threshold = threshold

    def display_conversation(self):
        '''display the conversation in a pretty format denoting the system, user and assistant differently'''
        for turn in self.conversation:
            role = turn['role']
            content = turn['content']
            if role == 'system':
                print(f'System: {content}')
            elif role == 'user':
                print(f'User: {content}')
            elif role == 'assistant':
                print(f'Assistant: {content}')
            print('------------')

    def user_turn(self, message):
        self.conversation.append({"role": "user", "content": message})
        best_result = get_best_result_from_pinecone(message)
        print(best_result)
        if best_result['score'] >= self.threshold:
            # Add to the context to the system prompt
            self.conversation[0]['content']+=f'\n\nFrom the knowledge base: "{best_result["text"]}"'

        chatgpt_response = openai_client.chat.completions.create(
            model='gpt-4',
            temperature=0,
            messages=self.conversation
        ).choices[0].message.content.strip()
        self.conversation.append({'role': 'assistant', 'content': chatgpt_response})
        return self.conversation[-1]


In [52]:
c = ChatbotGPT(system_prompt=SYSTEM_PROMPT)

c.user_turn('what are fixed costs?')

{'text': 'In economics, fixed costs, indirect costs or overheads are business expenses that are not dependent on the level of goods or services produced by the business. They tend to be time-related, such as salaries or rents being paid per month, and are often referred to as overhead costs. This is in contrast to variable costs, which are volume-related (and are paid per quantity produced). For a simple example, such as a bakery, the monthly rent for the baking facilities, and the monthly payments for the security system and basic phone line are fixed costs, as they do not change according to how much bread the bakery produces and sells. On the other hands, the wage costs of the bakery are variable, as the bakery will have to hire more workers if the production of bread increases. The relation between fixed cost and variable cost can be modelled by an analytical formula.', 'date_uploaded': '2023-08-14T15:04:37.247682', 'score': 0.88927865, 'id': '57a0103e5716168be7498b4531b21d07'}


{'role': 'assistant',
 'content': 'In economics, fixed costs are business expenses that are not dependent on the level of goods or services produced by the business. They tend to be time-related, such as salaries or rents being paid per month, and are often referred to as overhead costs. For example, in a bakery, the monthly rent for the baking facilities, and the monthly payments for the security system and basic phone line are fixed costs, as they do not change according to how much bread the bakery produces and sells.'}

In [53]:
c.user_turn('How old is Obama?')

{'text': "In November 2008, the show's post-election day telecast garnered the biggest audience in the show's history at 6.2 million in total viewers, becoming the week's most-watched program in daytime television. It was surpassed on July 29, 2010, during which former President Barack Obama first appeared as a guest on The View, which garnered a total of 6.6 million viewers. In 2013, the show was reported to be averaging 3.1 million daily viewers, which outpaced rival talk show The Talk.", 'date_uploaded': '2023-08-14T15:00:48.569014', 'score': 0.788732409, 'id': 'be0b9cb01bae3d7fa13839b85c10079c'}


{'role': 'assistant',
 'content': "I'm sorry, but I can't provide the answer to that question because the information is not in my knowledge base."}

In [54]:
c.user_turn('how many innings are in a game of baseball?')

{'text': "Ordinarily, a baseball game consists of nine innings (in softball and high school baseball games there are typically seven innings; in Little League Baseball, six), each of which is divided into halves: the visiting team bats first, after which the home team takes its turn at bat. However, if the score remains tied at the end of the regulation number of complete innings, the rules provide that ``play shall continue until (1) the visiting team has scored more total runs than the home team at the end of a completed inning; or (2) the home team scores the winning run in an uncompleted inning.'' (Since the home team bats second, condition (2) implies that the visiting team will not have the opportunity to score more runs before the end of the inning.)", 'date_uploaded': '2023-08-14T15:02:24.138099', 'score': 0.91325289, 'id': 'b2f88d00f95a155e6092e181a763ad03'}


{'role': 'assistant',
 'content': 'Ordinarily, a baseball game consists of nine innings. Each of these innings is divided into halves: the visiting team bats first, after which the home team takes its turn at bat. However, if the score remains tied at the end of the regulation number of complete innings, the rules provide that play shall continue until either the visiting team has scored more total runs than the home team at the end of a completed inning, or the home team scores the winning run in an uncompleted inning.'}

In [55]:
c.user_turn('thanks so much!')

{'text': "Susie moves on into another, larger part of heaven, occasionally watching earthbound events. Lindsey and Samuel have a daughter together named Abigail Suzanne. While stalking a young woman in New Hampshire, Harvey is hit on the shoulder by an icicle and falls to his death down a snow-covered slope into the ravine below. At the end of the novel, a Norristown couple finds Susie's charm bracelet but don't realize its significance, and Susie closes the story by wishing the reader ``a long and happy life''.", 'date_uploaded': '2023-08-14T15:03:33.926641', 'score': 0.736123919, 'id': '8ae1783aed03559f78a886c77e8001ff'}


{'role': 'assistant',
 'content': "You're welcome! If you have any other questions, feel free to ask."}

In [56]:
c.display_conversation()

System: You are a helpful Q/A bot that can only use reference material from a knowledge base.
If a user asks anything that is not "from the knowledge base", say that you cannot answer.

From the knowledge base: "In economics, fixed costs, indirect costs or overheads are business expenses that are not dependent on the level of goods or services produced by the business. They tend to be time-related, such as salaries or rents being paid per month, and are often referred to as overhead costs. This is in contrast to variable costs, which are volume-related (and are paid per quantity produced). For a simple example, such as a bakery, the monthly rent for the baking facilities, and the monthly payments for the security system and basic phone line are fixed costs, as they do not change according to how much bread the bakery produces and sells. On the other hands, the wage costs of the bakery are variable, as the bakery will have to hire more workers if the production of bread increases. The r